In [1]:
import cornac
import numpy as np
import pandas as pd
from cornac.eval_methods import RatioSplit


FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [2]:
train_df = pd.read_csv('../data/output/lsuo_train.csv', usecols=['userID', 'catID', 'like'])
validation_df = pd.read_csv('../data/output/lsuo_validation.csv', usecols=['userID', 'catID', 'like'])
test_df = pd.read_csv('../data/output/lsuo_test.csv', usecols=['userID', 'catID', 'like'])

train_df = train_df[['userID', 'catID', 'like']]
validation_df = validation_df[['userID', 'catID', 'like']]
test_df = test_df[['userID', 'catID', 'like']]

df = pd.concat([train_df, validation_df, test_df])

# make into tuples
data = list(df.itertuples(index=False, name=None))

# rearrange
for i in range(len(data)):
        data[i] = (str(data[i][1]), str(data[i][0]), float(data[i][2]))

In [3]:
train_users = train_df['userID'].unique()
test_users = test_df['userID'].unique()

In [4]:
both = np.intersect1d(test_users, train_users)
both

array([], dtype=object)

In [5]:
print(len(data))
print(type(data))
print(type(data[0]))
print(data[0])

4841
<class 'list'>
<class 'tuple'>
('3d2e4ecc-d9e0-48c9-b851-ebe8fdd9ede3', 'c5ab4a57-839d-40c9-ad03-70072da2d078', 1.0)


In [6]:
df.head()

,userID,catID,like
0,c5ab4a57-839d-40c9-ad03-70072da2d078,3d2e4ecc-d9e0-48c9-b851-ebe8fdd9ede3,True
1,c5ab4a57-839d-40c9-ad03-70072da2d078,20106676-a088-44e3-b23e-286b6e37f1d5,True
2,b2fe504c-8b80-4f50-ab07-12b6e7ff8cd2,54197c51-1b76-4854-9349-e2e2d55bd71d,False
3,1ef1a3c2-e71d-4df6-b89c-874697ca64d9,6e104908-8c00-45cd-ae3d-279dda0c3d9c,True
4,ab6872c5-4af8-4b92-95e2-b71363b2b82d,82838ab4-2ed8-4bc4-adf0-0d4e5edd9b92,False


In [7]:
filtered_train_df = train_df[train_df['like'] == True]
filtered_train_df.drop('like', axis=1, inplace=True)
filtered_train_df.head()

C:\Users\yongr\AppData\Local\Temp\ipykernel_35612\3561142669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df.drop('like', axis=1, inplace=True)


,userID,catID
0,c5ab4a57-839d-40c9-ad03-70072da2d078,3d2e4ecc-d9e0-48c9-b851-ebe8fdd9ede3
1,c5ab4a57-839d-40c9-ad03-70072da2d078,20106676-a088-44e3-b23e-286b6e37f1d5
3,1ef1a3c2-e71d-4df6-b89c-874697ca64d9,6e104908-8c00-45cd-ae3d-279dda0c3d9c
6,df3f9dd6-2538-4856-9137-ac08e9d6024d,7ef6aeea-738e-4064-b568-40a89431a4ee
7,c0d37cae-c1fc-4aca-9135-0e1d4f222a99,30c033c1-9ebf-47de-ba85-bb2ca1029ed9


In [8]:
filtered_validation_df = validation_df[validation_df['like'] == True]
filtered_validation_df.drop('like', axis=1, inplace=True)
filtered_validation_df.head()

C:\Users\yongr\AppData\Local\Temp\ipykernel_35612\2641804607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_validation_df.drop('like', axis=1, inplace=True)


,userID,catID
1,081f358f-9624-468a-b6ef-f9b9fad2b3b3,2b87337b-1179-42c8-b423-c0ec34bb6833
3,7031fead-a8c9-4702-a610-2873cb67f8a1,c4c51825-42a2-4ffe-8096-d3d0200df200
5,64515f3e-a2c6-4f08-8dd7-5e79116f07be,a60e85ad-1f8e-46c2-b659-ebc27ae94966
7,dea8ada2-c94b-4b8c-b2de-8d94586f7ffc,c822a63a-ad76-4302-af0a-ef17f4ae3680
11,081f358f-9624-468a-b6ef-f9b9fad2b3b3,66c72d11-c009-4ff1-b976-353aa1072cac


In [9]:
# !pip install recommenders
# train_df and validation_df is passed in currently as a POC, not exactly supposed to pass in these

from recommenders.evaluation.python_evaluation import serendipity

eval_serendipity = serendipity(train_df, validation_df, col_user='userID', col_item='catID',
                               col_relevance='like')
print(eval_serendipity)

nan


In [10]:
from harmonic_mean import HarmonicMean
from serendipity_wrapper import Serendipity
from combined_eval_method import CombinedBaseMethod

# Define an evaluation method to split feedback into train and test sets
ratio_split = CombinedBaseMethod.from_splits(
    train_data=train_df.values,
    val_data=validation_df.values,
    test_data=test_df.values,
    rating_threshold=1.0,
    # exclude_unknowns=True,
    verbose=True,
)

# Instantiate the most popular baseline, BPR model
most_pop = cornac.models.MostPop()
bpr = cornac.models.BPR(
    k=50, max_iter=200, learning_rate=0.001, lambda_reg=0.001, verbose=True
)

# Use these for evaluation
eval_metrics = [
    HarmonicMean(
        10,
        Serendipity(),
        cornac.metrics.FMeasure(k=10),
        cornac.metrics.NCRR(),
        cornac.metrics.NDCG()
    ),
    Serendipity(),
    cornac.metrics.FMeasure(k=10),
    cornac.metrics.NCRR(),
    cornac.metrics.NDCG()
]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[most_pop, bpr],
    metrics=eval_metrics,
).run()

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[MostPop] Training started!

[MostPop] Evaluation started!


Ranking:   0%|          | 0/11 [00:00<?, ?it/s]

Ranking:   0%|          | 0/10 [00:00<?, ?it/s]


[BPR] Training started!


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!

[BPR] Evaluation started!


Ranking:   0%|          | 0/11 [00:00<?, ?it/s]

Ranking:   0%|          | 0/10 [00:00<?, ?it/s]


VALIDATION:
...
        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------- + ------ + ------------ + ------- + ------- + ----------- + --------
MostPop | 0.0706 |       0.0000 |  0.1958 |  0.4697 |      0.0000 |   0.5024
BPR     | 0.0461 |       0.0000 |  0.1887 |  0.4657 |      0.0000 |   0.5120

TEST:
...
        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
MostPop | 0.0764 |       0.0000 |  0.2282 |  0.4813 |      0.0000 |    0.0012 |   0.4872
BPR     | 0.0672 |       0.0000 |  0.2268 |  0.4811 |      0.0000 |    0.2256 |   0.5099

